In [1]:
import numpy as np

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, GRU
from keras.callbacks import ModelCheckpoint
from random import randint
from keras.models import model_from_json


In [2]:
with open("hazes.txt") as corpus_file:
    corpus = corpus_file.read()
    corpus = corpus.lower()
print("Loaded a corpus of {0} characters".format(len(corpus)))

Loaded a corpus of 41701 characters


In [3]:
from keras.models import model_from_json

# Build our network from loaded architecture and weights

from tensorflow import keras

with open('model.json', 'r') as file:
    data = file.read()

model = model_from_json(data)


model.load_weights('weights-20.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               86016     
                                                                 
 dense (Dense)               (None, 39)                5031      
                                                                 
 activation (Activation)     (None, 39)                0         
                                                                 
Total params: 91,047
Trainable params: 91,047
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Get a unique identifier for each char in the corpus, then make some dicts to ease encoding and decoding

# Get the unique characters in the corpus
# sort them in alphabetical order
chars = sorted(list(set(corpus)))

# Set the length of the input sentence
sentence_length = 20

# Get the total number of unique characters
num_chars = len(chars)

# Create a dictionary that maps each character to a numerical value
encoding = {c: i for i, c in enumerate(chars)}

# Create a dictionary that maps each numerical value back to its corresponding character
decoding = {i: c for i, c in enumerate(chars)}


In [5]:
def sample(preds, temperature=1.0):
  if temperature <= 0:
    return np.argmax(preds)
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [6]:
def generate(seed_pattern):
        X = np.zeros((1, sentence_length, num_chars), dtype=bool)
        #print(X.shape)
        for i, character in enumerate(seed_pattern):
            X[0, i, encoding[character]] = 1
        
        generated_text = ""
        for i in range(500):
            # even de temperatuur toevoegen.
            prediction = sample(model.predict(X, verbose=0)[0],0.5)
            generated_text += decoding[prediction]

            activations = np.zeros((1, 1, num_chars), dtype=bool)
            activations[0, 0, prediction] = 1
            X = np.concatenate((X[:, 1:, :], activations), axis=1)

        return generated_text

def make_seed(seed_phrase=""):
        if seed_phrase:
            phrase_length = len(seed_phrase)
            pattern = ""
            for i in range (0, sentence_length):
                pattern += seed_phrase[i % phrase_length]
        else:
            seed = randint(0, corpus_length - sentence_length)
            pattern = abba_corpus[seed:seed + sentence_length]

        return pattern

In [7]:
sentence_length = 20
seed = make_seed('ik geloof in mezelf ')
print(seed)
txt =  generate(seed)
print(txt)

ik geloof in mezelf 
oh jij alls al geweld

je heb je zijn deer staan
in deroof jij naar dij welder
dat ik echt niet meer
je haar techund gaan
wat ik jou niet kijkt bedomen
en wiet ik mij zo verden
ik hou van jou, maar ik heb het zenden
maar ja ik zo dan had
met is je nooit maar ik weer een stouwen gedacht
als je diekelen
want vengen lieg, ook is het voorbij,

joh, ja aat je vrienden anner
als zij ik mijn hielper te van messt er kurste hag
je blijft toel verdeel veeg ouwer jou gezelt
ik wil je niet meer dat vergelez


## Sample song
In the bard and show you on your lovelight and i can't get the mowner i'm a marion an and every mind, there's a boot